In [1]:
import matplotlib.pyplot as plt
import pickle, gzip
import numpy as np

import sys
sys.path.append('../src')

import neuralnetworkclassifier as nnc
import mlutils as ml

In [2]:
!wget http://deeplearning.net/data/mnist/mnist.pkl.gz

--2019-11-07 21:59:34--  http://deeplearning.net/data/mnist/mnist.pkl.gz
Resolving deeplearning.net (deeplearning.net)... 132.204.26.28
Connecting to deeplearning.net (deeplearning.net)|132.204.26.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16168813 (15M) [application/x-gzip]
Saving to: ‘mnist.pkl.gz’

mnist.pkl.gz        100%[===================>]  15.42M   796KB/s    in 20s     

2019-11-07 21:59:54 (778 KB/s) - ‘mnist.pkl.gz’ saved [16168813/16168813]



In [3]:
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

Xtrain = train_set[0].reshape(-1, 1, 28, 28)
Ttrain = train_set[1].reshape((-1, 1))

Xtest  = test_set[0].reshape(-1, 1, 28, 28)
Ttest  = test_set[1].reshape((-1, 1))

print(Xtrain.shape, Ttrain.shape, Xtest.shape, Ttest.shape)

(50000, 1, 28, 28) (50000, 1) (10000, 1, 28, 28) (10000, 1)


In [4]:
nnet = nnc.NeuralNetwork_Convolutional(n_channels_in_image=Xtrain.shape[1],
                                       image_size=Xtrain.shape[2],
                                       n_units_in_conv_layers=[10], 
                                       kernels_size_and_stride=[(7, 3)],
                                       n_units_in_fc_hidden_layers=[],
                                       classes=np.unique(Ttrain), use_gpu=True)

nnet.train(Xtrain, Ttrain, 100, optim='Adam', learning_rate=0.05, verbose=True)


GPU is not available. Running on CPU.

Epoch 10 error 0.55505
Epoch 20 error 0.32390
Epoch 30 error 0.21339
Epoch 40 error 0.15725
Epoch 50 error 0.12479
Epoch 60 error 0.10390
Epoch 70 error 0.08996
Epoch 80 error 0.07966
Epoch 90 error 0.07155
Epoch 100 error 0.06485


In [5]:
nnet

NeuralNetwork_Convolutional(
                            n_channels_in_image=1,
                            image_size=28,
                            n_units_in_conv_layers=[10],
                            kernels_size_and_stride=[(7, 3)],
                            n_units_in_fc_hidden_layers=[],
                            classes=[0 1 2 3 4 5 6 7 8 9],
                            use_gpu=False)
Sequential(
  (conv_0): Conv2d(1, 10, kernel_size=(7, 7), stride=(3, 3))
  (output_0): Tanh()
  (flatten): Flatten()
  (output_1): Linear(in_features=640, out_features=10, bias=True)
)
   Network was trained for 100 epochs that took 192.7188 seconds.
   Final objective value is 0.065

In [6]:
train_percent = ml.percent_correct(Ttrain, nnet.use(Xtrain)[0])

classes, Y    = nnet.use(Xtest)
test_percent  = ml.percent_correct(Ttest, classes)

print(f'Train Percentage: {train_percent}, Test Percentage: {test_percent}')

Train Percentage: 98.156, Test Percentage: 96.8


In [7]:
_ = ml.confusion_matrix(Ttest, classes, np.unique(Ttest))

       0    1    2    3    4    5    6    7    8    9
    ------------------------------------------------------------
 0 | 98.8  0.1  0.1  0    0    0.1  0.4  0.2  0.3  0  
 1 |  0   99.0  0.2  0.1  0    0    0.3  0.1  0.4  0  
 2 |  0.6  0.4 95.7  0.6  0.2  0    0.2  0.5  1.5  0.4
 3 |  0.1  0    0.6 97.2  0.1  1.0  0    0.4  0.4  0.2
 4 |  0.1  0.1  0.2  0   97.4  0    0.6  0.1  0.2  1.3
 5 |  0.2  0.1  0.1  0.9  0   97.0  0.6  0.1  1.0  0  
 6 |  0.5  0.3  0.4  0.1  0.4  0.8 97.2  0    0.2  0  
 7 |  0    0.6  1.2  0.6  0.6  0.2  0   95.3  0.1  1.5
 8 |  0.8  0.1  0.7  0.7  0.4  0.6  0.2  0.7 95.0  0.7
 9 |  0.5  0.4  0    0.7  1.4  0.8  0    0.8  0.2 95.2
